<a href="https://colab.research.google.com/github/denisefss/Mapping_susceptibility_landslides_BH/blob/main/Extracao_pixel_raster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook explica como extrair os dados das variáveis raster preditoras para a camada de pontos com pontos de presença/ausência de deslizamentos

In [ ]:
# Montar o google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalar bibliotecas necessárias
!pip install rasterio
!pip install pyspatialml
!pip install numpy

In [ ]:
# Importar as bibliotecas necessárias
from pyspatialml import Raster
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import seaborn as sn
import rasterio.plot
import numpy as np

In [ ]:
# Alterar o diretório de trabalho
os.chdir(r'/caminho_do_seus_arquivos_no_drive')

In [ ]:
# Carregar a camada de pontos com presença/ausência de deslizamentos
# Substitua pelo nome real do seu arquivo shapefile
points_path = 'dados.shp'
points_gdf = gpd.read_file(points_path)

In [ ]:
# Substitua pelos nomes dos seus rasters de saída
predictors = ['SPI_bh.tif', 'TPI_bh.tif', 'TRI_bh.tif', 'TWI_bh.tif', 'declividade_bh.tif','densi_hidro.tif', 'densi_lineamento_bh.tif','distancia_hidro_bh.tif',
              'distancia_lineamento_bh.tif', 'distancia_vias_bh.tif', 'elevacao_bh.tif', 'flow_acc_bh.tif',
              'lito_bh.tif','modelo_cine.tif', 'ndvi_resized.tif', 'orientacao_bh.tif', 'pedo_bh.tif',
              'planform_curvature_bh.tif','pot_uso_bh.tif','profile_curvature_bh.tif', 'rugosidade_bh.tif', 'unid_geo_bh.tif', 'densi_vias.tif']

In [ ]:
# Criar uma pilha com os rasters preditores
stack = Raster(predictors)

In [ ]:
# Acessando os nomes dos rasters
names = stack.names
# Imprimindo os nomes dos rasters
print("Nomes dos rasters no stack:", names)

In [ ]:
points_gdf= points_gdf[['id', 'Target', 'geometry']]

In [ ]:
# Substitua 'seu_arquivo.shp' pelo caminho do seu arquivo shapefile
# Lê o shapefile
gdf = points_gdf

# Lista para armazenar os valores de pixel
pixel_values = []

# Loop sobre cada raster na pilha
for raster_file in predictors:
    with rasterio.open(raster_file) as src:
        # Reprojeta o shapefile para o sistema de coordenadas do raster
        gdf_reproj = gdf.to_crs(src.crs)

        # Obtém as coordenadas dos pontos
        coords = [(point.x, point.y) for point in gdf_reproj.geometry]

        # Converte as coordenadas para índices de linha e coluna no raster
        rows, cols = zip(*[src.index(x, y) for x, y in coords])

        # Lê os valores de pixel nos pontos
        values = src.read(1)  # Lê a banda 1 (ou ajuste conforme necessário)
        pixel_values.append(values[rows, cols])

# Converte a lista de valores de pixel em um array numpy
pixel_values_array = np.array(pixel_values).T

# Adiciona os valores de pixel ao GeoDataFrame original sem o sufixo "_value"
for i, raster_name in enumerate(predictors):
    gdf[raster_name.replace('.tif', '')] = pixel_values_array[:, i]

# Exibe o GeoDataFrame resultante com os valores de pixel adicionados
print(gdf)

In [ ]:
gdf.to_csv('dados_pixel.csv', index = False)